In [ ]:
import datetime as dt
from glob import glob
from tqdm import tqdm
import json
import pandas as pd
import re
import csv
import math

## 成熟後かつ3ヶ月の期間が被っていないバージョン
project_list = ['Glance', 'Swift', 'Keystone', 'Cinder', 'Neutron', 'Nova']
dates_list = [[dt.datetime(2015,6,18), dt.datetime(2015,12,2), dt.datetime(2016,3,3), dt.datetime(2016,6,1), dt.datetime(2018,4,19)],
              [dt.datetime(2013,8,20), dt.datetime(2015,9,1), dt.datetime(2016,3,25), dt.datetime(2018,2,5), dt.datetime(2021,3,18)],
              [dt.datetime(2015,6,17), dt.datetime(2016,3,3), dt.datetime(2016,7,14), dt.datetime(2016,11,17), dt.datetime(2019,3,21)],
              [dt.datetime(2015,12,3), dt.datetime(2016,3,17), dt.datetime(2016,7,14), dt.datetime(2017,6,8), dt.datetime(2017,12,6)],
              [dt.datetime(2015,6,24), dt.datetime(2015,12,3), dt.datetime(2016,6,3), dt.datetime(2017,4,14), dt.datetime(2020,2,22)],
              [dt.datetime(2016,3,3), dt.datetime(2016,6,2), dt.datetime(2017,6,8), dt.datetime(2017,10,24), dt.datetime(2019,9,27)]]

time_delta = dt.timedelta(weeks=12)
for project, dates in tqdm(zip(project_list, dates_list)):
    filepath_list = glob(f'/Users/mizuki-u/current/dataset/{project}/list_1/*.json')
    review_count_list = []
    for date in dates:
        start_date = (date - time_delta).date()
        end_date = date.date()
        for filepath in filepath_list:
            with open(filepath) as f:
                json_load = json.load(f)
            message_list = json_load['messages']
            for msg in message_list:
                date = pd.to_datetime(msg['date']).normalize()
                if start_date < date < end_date:
                    if any(['Looks good to me, approved' in msg['message'], 'Looks good to me, but someone else must approve' in msg['message'],
                            'I would prefer this is not submitted as is' in msg['message'], 'This shall not be submitted' in msg['message'],
                            'Code-Review+2' in msg['message'], 'Code-Review+1' in msg['message'], 'Code-Review-1' in msg['message'],
                            'Code-Review-2' in msg['message']]):
                        flag = True
                        for rev_list in review_count_list:
                            if msg['author']['name'] == rev_list[0]:
                                rev_list[1] += 1
                                flag = False
                        if flag:
                            review_count_list.append([msg['author']['name'], 1])
                elif end_date < date:
                    break
    
    sorted_review_count_list = sorted(review_count_list, key=lambda x: x[1], reverse=True)
    df = pd.DataFrame(sorted_review_count_list[0:10], columns=['name', 'review_count'])
    df.to_csv(f'/Users/mizuki-u/current/論文誌用/{project}/review_count.csv', index=False)

In [16]:
from glob import glob
from tqdm import tqdm
import json
import pandas as pd
import csv

project_list = ['Glance', 'Swift', 'Keystone', 'Cinder', 'Neutron', 'Nova']
for project in tqdm(project_list):
    filepath_list = glob(f'/Users/mizuki-u/current/dataset/{project}/list_1/*.json')
    review_mail_list = []
    with open(f'/Users/mizuki-u/current/論文誌用/{project}/review_count.csv') as f:
        reader = csv.DictReader(f)
        reviewer_line = [row for row in reader]
    for reviewer in reviewer_line:
        reviewer_name = reviewer['name']
        flag = False
        for filepath in filepath_list:
            with open(filepath) as f:
                json_load = json.load(f)
            message_list = json_load['messages']
            for msg in message_list:
                if any(['Looks good to me, approved' in msg['message'], 'Looks good to me, but someone else must approve' in msg['message'],
                        'I would prefer this is not submitted as is' in msg['message'], 'This shall not be submitted' in msg['message'],
                        'Code-Review+2' in msg['message'], 'Code-Review+1' in msg['message'], 'Code-Review-1' in msg['message'],
                        'Code-Review-2' in msg['message']]):
                    try:
                        if msg['author']['name'] == reviewer_name:
                            review_mail_list.append(msg['author']['email'])
                            flag = True
                            break
                    except KeyError:
                        pass
            if flag:
                break
    df = pd.DataFrame(review_mail_list, columns=['mail'])
    df.to_csv(f'/Users/mizuki-u/current/論文誌用/{project}/reviewer_mail.csv', index=False)

100%|██████████| 6/6 [00:00<00:00,  8.73it/s]


In [17]:
from glob import glob
from tqdm import tqdm
import json
import pandas as pd
import csv

project_list = ['Glance', 'Swift', 'Keystone', 'Cinder', 'Neutron', 'Nova']
for project in tqdm(project_list):
    filepath_list = glob(f'/Users/mizuki-u/current/dataset/{project}/list_1/*.json')
    review_mail_list = [[],[],[],[],[],[],[],[],[],[]]
    with open(f'/Users/mizuki-u/current/論文誌用/{project}/review_count.csv') as f:
        reader = csv.DictReader(f)
        reviewer_line = [row for row in reader]
    for idx, reviewer in enumerate(reviewer_line):
        reviewer_name = reviewer['name']
        for filepath in filepath_list:
            with open(filepath) as f:
                json_load = json.load(f)
            message_list = json_load['messages']
            for msg in message_list:
                if any(['Looks good to me, approved' in msg['message'], 'Looks good to me, but someone else must approve' in msg['message'],
                        'I would prefer this is not submitted as is' in msg['message'], 'This shall not be submitted' in msg['message'],
                        'Code-Review+2' in msg['message'], 'Code-Review+1' in msg['message'], 'Code-Review-1' in msg['message'],
                        'Code-Review-2' in msg['message']]):
                    try:
                        if msg['author']['name'] == reviewer_name and msg['author']['email'] not in review_mail_list[idx]:
                            review_mail_list[idx].append(msg['author']['email'])
                    except KeyError:
                        pass
    print(review_mail_list)
    # sorted_review_count_list = sorted(review_count_list, key=lambda x: x[1], reverse=True)
    # df = pd.DataFrame(sorted_review_count_list[0:10], columns=['name', 'review_count'])
    # df.to_csv(f'/Users/mizuki-u/current/論文誌用/{project}/review_count.csv', index=False)

 17%|█▋        | 1/6 [00:10<00:51, 10.23s/it]

[['fpercoco@redhat.com'], ['jokke@usr.fi'], ['kkushaev@mirantis.com'], ['nik.komawar@gmail.com'], ['akekane@redhat.com'], ['mfedosin@redhat.com'], ['rosmaita.fossdev@gmail.com'], ['sigmavirus24@gmail.com'], ['stuart.mclaren@hpe.com'], ['damalyavkina@gmail.com']]


 33%|███▎      | 2/6 [00:23<00:48, 12.19s/it]

[['clay.gerrard@gmail.com'], ['alistairncoles@gmail.com'], ['sam@swiftstack.com'], ['tburke@nvidia.com'], ['matt@oliver.net.au'], ['zaitcev@kotori.zaitcev.us'], ['cschwede@redhat.com'], ['me@not.mn'], ['bloodeagle40234@gmail.com'], ['thiagodasilva@gmail.com']]


 50%|█████     | 3/6 [00:41<00:44, 14.78s/it]

...

[['sean.mcginnis@gmail.com'], ['jungleboyj@electronicjungle.net'], ['xingyang105@gmail.com'], ['geguileo@redhat.com'], ['e0ne@e0ne.info'], ['mdulko@redhat.com'], ['eharney@redhat.com'], ['kennelson11@gmail.com'], ['john.griffith8@gmail.com'], ['waboring@hemna.com']]


 83%|████████▎ | 5/6 [05:29<01:33, 93.98s/it]

[['ihrachys@redhat.com'], ['kevin@benton.pub'], ['gkotton@vmware.com'], ['armamig@gmail.com'], ['yamamoto@midokura.com'], ['haleyb.dev@gmail.com'], ['ichihara.hirofumi@gmail.com'], ['zzelle@gmail.com'], ['carl@ecbaldwin.net'], ['vikschw@gmail.com']]


100%|██████████| 6/6 [09:05<00:00, 90.97s/it] 

[['mriedem.os@gmail.com'], ['sean@dague.net'], ['stephenfin@redhat.com'], ['john@johngarbutt.com'], ['takanattie@gmail.com'], ['dms@danplanet.com'], ['ken1ohmichi@gmail.com'], ['openstack@fried.cc'], ['hejie.xu@intel.com'], ['cdent@anticdent.org']]


In [7]:
from glob import glob
from tqdm import tqdm
import json
import pandas as pd
import csv

project_list = ['Glance', 'Swift', 'Keystone', 'Cinder', 'Neutron', 'Nova']
list1 = []
for project in tqdm(project_list):
    with open(f'/Users/mizuki-u/current/論文誌用/{project}/review_count.csv') as f:
        reader = csv.DictReader(f)
        reviewer_line = [row['name'] for row in reader]
        list1.extend(reviewer_line)
print(len(list(set(list1))))

100%|██████████| 6/6 [00:00<00:00, 3218.96it/s]

60
